In [2]:
# import sys
# !{sys.executable} -m pip install fpdf -q 
# !{sys.executable} -m pip install pandas -q
# !{sys.executable} -m pip install plotly-express -q
# !{sys.executable} -m pip install kaleido -q



In [3]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd 
import plotly.express as px
from fpdf import FPDF

# Defind paths and chart style

In [4]:
# Define the plotly template. Some other examples:
# "plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"
plotly_template = "presentation"

In [6]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directories if they don't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Total Sales by Month

In [7]:
# Create a connection to the database
conn = sqlite3.connect(database_path)

In [8]:
# Execute the query and load the results into a pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) AS total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [9]:
# Print the DataFrame
print(df)

      sale_date  total_sales
0    2022-01-01        23.76
1    2022-01-02        23.76
2    2022-01-03        23.76
3    2022-01-04        16.83
4    2022-01-05        12.87
..          ...          ...
341  2022-12-27         7.92
342  2022-12-28         8.91
343  2022-12-29        18.81
344  2022-12-30         6.93
345  2022-12-31        15.84

[346 rows x 2 columns]


In [10]:
# check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sale_date    346 non-null    object 
 1   total_sales  346 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.5+ KB


In [11]:
# Convert the sale_date column to a datetime object
df["sale_date"] = pd.to_datetime(df["sale_date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    346 non-null    datetime64[ns]
 1   total_sales  346 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.5 KB


In [12]:
# Set the sale_date column as the index
df = df.set_index("sale_date")
df.head(3)

,total_sales
sale_date,
2022-01-01,23.76
2022-01-02,23.76
2022-01-03,23.76


In [13]:
# Resample the data to a monthly frequency and compute the sum
df_monthly = df.resample("M").sum()
df_monthly

,total_sales
sale_date,
2022-01-31,436.59
2022-02-28,341.55
2022-03-31,480.15
2022-04-30,445.50
2022-05-31,417.78
2022-06-30,476.19
2022-07-31,423.72
2022-08-31,504.90
2022-09-30,466.29


In [14]:
# Map the month number to short month name
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,436.59,Jan
2022-02-28,341.55,Feb
2022-03-31,480.15,Mar
2022-04-30,445.50,Apr
2022-05-31,417.78,May
2022-06-30,476.19,Jun
2022-07-31,423.72,Jul
2022-08-31,504.90,Aug
2022-09-30,466.29,Sep


In [15]:
# Create the Plotly figure with text parameter
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title="Total Sales by Month",
    xaxis_title="Month",
    yaxis_title="Total Sales ($)",
    yaxis_tickprefix="$",
)

# Show the figure
fig.show()

# Save the figure as a PNG file
fig.write_image(output_dir / "total_sales_by_month.png",
                width=1200,
                height=400,
                scale=4)

# Total Sales by Product

In [21]:
# Execute the query and load results into a Panda DataFrame
query = '''
SELECT p.product_name, SUM(s.total_price) AS total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''

df = pd.read_sql_query(query, conn)

In [22]:
df

,product_name,total_sales
0,Apple,79.20
1,Asparagus,71.28
2,Avocado,139.59
3,Banana,106.92
4,Beans,63.36
5,Beet,171.27
6,Blackberry,112.86
7,Blueberry,82.17
8,Broccoli,88.11
9,Brussel Sprout,131.67


In [25]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title="Total Sales by Product",
    xaxis_title="Product",
    yaxis_title="Total Sales ($)",
    yaxis_tickprefix="$",
)

# Show the figure
fig.show()

# Save the figure as a PNG file
fig.write_image(output_dir / "total_sales_by_product.png",
                width=1200,
                height=400,
                scale=4)

                

# Top Customer by Sales

In [26]:
# Execute the query and load results into a Panda DataFrame
query = ''' 
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
''' 

df = pd.read_sql_query(query, conn)

In [27]:
df 

,customer_name,total_sales
0,Bob Smith,642.51
1,Jane Doe,629.64
2,Alice Jones,616.77
3,Frank Wilson,613.80
4,Emily Davis,516.78
5,David Brown,507.87
6,John Doe,493.02
7,Isabel Garcia,469.26
8,Grace Lee,431.64
9,Henry Chen,405.90


In [29]:
# Create the Plotly figure with text parameter
fig = px.bar(df,
             x='customer_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

# Set the layout
fig.update_layout(
    title="Top 10 Customers by Total Sales",
    xaxis_title="Customer",
    yaxis_title="Total Sales ($)",
    yaxis_tickprefix="$",
)

# Show the figure
fig.show()

# Save the figure as a PNG file
fig.write_image(output_dir / "top_10_customers_by_total_sales.png",
                width=1200,
                height=400,
                scale=4)



# Create PDF Report

In [34]:
# Define the font color as RGB values (dark gray)
font_color = (64, 64, 64)

# Find all PNG files in the output directory
chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

# Create a PDF document and set the page size 
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

# Add the overall page title
title = f"Sales Report as of {date.today().strftime('%d/%m/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='Cl',ln= 1)

# Add each chart to the PDF document
for chart_filename in chart_filenames:
    pdf.ln(10) # add padding at the top of the next chart
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h = 0)
    
    
# Save the PDF document
pdf.output(output_dir / "sales_report.pdf", "F")


''

# Bonus - SQL vs. Pandas

## Customer Segment using SQL

In [38]:
# Execute the query and load results into a Panda DataFrame
query =  ''' 
SELECT
    customers.customer_id,
    customers.first_name || ' ' || customers.last_name AS customer_name,
    SUM(sales.total_price) AS total_sales,
    CASE
        WHEN SUM(sales.total_price) > 450 then 'High Value'
        WHEN SUM(sales.total_price) > 200 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
''' 

df = pd.read_sql_query(query, conn)
df 


,customer_id,customer_name,total_sales,customer_segment
0,3,Bob Smith,642.51,High Value
1,2,Jane Doe,629.64,High Value
2,4,Alice Jones,616.77,High Value
3,7,Frank Wilson,613.80,High Value
4,6,Emily Davis,516.78,High Value
5,5,David Brown,507.87,High Value
6,1,John Doe,493.02,High Value
7,10,Isabel Garcia,469.26,High Value
8,8,Grace Lee,431.64,Medium Value
9,9,Henry Chen,405.90,Medium Value


## Customer Segment using SQL + Pandas

In [39]:
# Execute the query and load results into a Panda DataFrame
query = '''
SELECT
    customers.customer_id,
    customers.first_name || ' ' || customers.last_name AS customer_name,
    SUM(sales.total_price) AS total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
''' 

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales
0,1,John Doe,493.02
1,2,Jane Doe,629.64
2,3,Bob Smith,642.51
3,4,Alice Jones,616.77
4,5,David Brown,507.87
5,6,Emily Davis,516.78
6,7,Frank Wilson,613.80
7,8,Grace Lee,431.64
8,9,Henry Chen,405.90
9,10,Isabel Garcia,469.26


In [41]:
# Group the customers into segments
bins = [0, 200, 450, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df 

,customer_id,customer_name,total_sales,customer_segment
2,3,Bob Smith,642.51,High Value
1,2,Jane Doe,629.64,High Value
3,4,Alice Jones,616.77,High Value
6,7,Frank Wilson,613.80,High Value
5,6,Emily Davis,516.78,High Value
4,5,David Brown,507.87,High Value
0,1,John Doe,493.02,High Value
9,10,Isabel Garcia,469.26,High Value
7,8,Grace Lee,431.64,Medium Value
8,9,Henry Chen,405.90,Medium Value


In [42]:
# Close the database connection
conn.close()